# Planning and Stock Report Generation

---

## Pre-requisites

- Create conda env using the following command from the root of this repo: `conda env create -f ./notebooks/multi-agent-systems/pyautogen/conda-env.yaml`.
- Please define the following environment variables in `.env` file:
    - `OPENAI_API_BASE="<ENDPOINT_URL>"`
    - `OPENAI_API_KEY="<API_KEY>"`

## Introduction

Here, we wil learn about **multi-agent group chats** where we will build a custom group chat that collaborates to generate a detailed report about stock performance over the past month. Multi-agents group chat allows us to define agents without specifying any fixed execution order. This allows us to create a more flexible and dynamic system, where agents can interact with each other in a more complex way by chiming in whenever required.

<img src="./images/group-chat-pattern.png" alt="Group Chat Pattern" style="width: 30%;"/>

## Setup

In [1]:
import warnings
import os
from dotenv import load_dotenv, find_dotenv

warnings.filterwarnings('ignore')
_ = load_dotenv(find_dotenv())

Please find additional info about how to create LLM Configuration [over here](https://microsoft.github.io/autogen/docs/topics/llm_configuration).

In [2]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_type": "azure",
    "base_url": os.environ.get("OPENAI_API_BASE"),
    "api_key": os.environ.get("OPENAI_API_KEY"),
}

## The task!

In [3]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-08-26."

## Build a group chat

This group chat will include these agents:
- **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
- **Planner**: to determine relevant information needed to complete the task.
- **Engineer**: to write code using the defined plan by the planner.
- **Executor**: to execute the code written by the engineer.
- **Writer**: to write the report.

In [4]:
import autogen

In [5]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [6]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [7]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

> Note: In this lesson, we'll use an alternative method of code execution by providing a dict config. However, we can also use the `LocalCommandLineCodeExecutor`.

In [8]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [9]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [10]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [11]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

In [12]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-08-26.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, we'll need to gather and analyze specific data. Here's the plan:

1. **Retrieve Stock Price Data**: Obtain daily stock prices of Nvidia from July 26, 2024, to August 26, 2024. This should include opening, closing, high, and low prices.

2. **Calculate Performance Metrics**:
   - Measure the percentage change from the beginning of the period to the end.
   - Identify the highest and lowest prices during this period and when they occurred.
   - Calculate average closing price.

3. **Visualize Data**:
   - Prepare visualizations such as line graphs showing the price movement over the month.

4. **Conduct Comparati

## Add a speaker selection policy

In [13]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [17]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [18]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-08-26.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia in the past month, we'll need to gather and analyze specific data. Here's the plan:

1. **Retrieve Stock Price Data**: Obtain daily stock prices of Nvidia from July 26, 2024, to August 26, 2024. This should include opening, closing, high, and low prices.

2. **Calculate Performance Metrics**:
   - Measure the percentage change from the beginning of the period to the end.
   - Identify the highest and lowest prices during this period and when they occurred.
   - Calculate average closing price.

3. **Visualize Data**:
   - Prepare visualizations such as line graphs showing the price movement over the month.

4. **Conduct Comparati